In [1]:
import sys
sys.path.append('../')
import torch
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from models_control import NetNormalDropoutV2
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim import Adam

### Control experiment: Not using masks and no dropout
This notebook evaluates the performance oof using a standard deep-net with 'normal' dropout layers in training which are removed at inferene

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(), 
])

In [3]:
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)


In [4]:
BATCH_SIZE = 32
EPOCHS = 12
NUM_MASKS = 1
LR = 0.001

In [5]:
seed = 42
torch.manual_seed(seed)
train_dataloader = DataLoader(dataset1, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset2)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = NetNormalDropoutV2()
opt = Adam(model.parameters(), lr=LR)
lossFn = torch.nn.NLLLoss() # Use NLL since we our model is outputting a probability


In [7]:
for i in range(EPOCHS):
    model.train()
    trainCorrect = 0
    totalLoss = 0
    for idx, (x, y)  in tqdm(enumerate(train_dataloader)):
        x, y = x.to(device), y.to(device)
        logits = model.forward(x)
        loss = lossFn(logits, y)
        totalLoss += loss.item()
        opt.zero_grad()
        loss.backward()
        opt.step()
        trainCorrect += (logits.argmax(1) == y).type(
			torch.float).sum().item()
    print(f"Train Accuracy: {trainCorrect/len(dataset1)}")
    print(f"Total loss: {totalLoss}")

0it [00:00, ?it/s]

/var/folders/5q/7b4nn19d1rj3vsg11yxvc8lw0000gn/T/ipykernel_5943/2326333523.py:9: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Scalar.cpp:23.)
  totalLoss += loss.item()
1875it [00:54, 34.58it/s]


Train Accuracy: 0.9237333333333333
Total loss: 471.73156935488805


1875it [00:53, 35.11it/s]


Train Accuracy: 0.9685
Total loss: 198.8039915394038


...

Train Accuracy: 0.9890666666666666
Total loss: 67.28966522109113


1875it [00:52, 35.47it/s]


Train Accuracy: 0.98955
Total loss: 62.40824907167007


1875it [00:52, 35.59it/s]

Train Accuracy: 0.9905166666666667
Total loss: 55.64953193461406


In [9]:
test_correct = 0
model.eval()
for idx, (x, y)  in tqdm(enumerate(test_dataloader)):
    logits = model.forward(x)
    pred = torch.argmax(logits, dim=1)
    test_correct += (pred == y).sum().item()
print(test_correct / len(dataset2))

10000it [00:02, 4584.16it/s]

0.9583
